In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)
import seaborn as sns
import plotly.express as px
import os
import missingno as msno
import pickle
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import matplotlib

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler

In [ ]:
orders = pd.read_csv('orders.csv')
deliveries = pd.read_csv('deliveries.csv')

def beg_end_month(x):
    if x<=10:
        return '월초'
    elif 10<x<=20:
        return '월중'
    elif 20<x<=31:
        return '월말'
orders.BKG_DATE = pd.to_datetime(orders.BKG_DATE, format='%Y-%m-%d')
orders.INS_DATE = pd.to_datetime(orders.INS_DATE, format='%Y-%m-%d')
orders["BKG_TIME"] = pd.to_datetime(orders["BKG_TIME"], format='%Y-%m-%d %H:%M:%S')
orders['BKG_WEEK'] = orders.BKG_DATE.dt.week
orders['BKG_MONTH2'] = orders.BKG_DATE.dt.day.map(beg_end_month)

In [ ]:
data = orders[orders.BKG_TYP==7][orders.CORP_ID=='KX007'].groupby(['BKG_DATE','BKG_HOUR'])['ITEM_QTY'].sum().reset_index()
data = data.append(pd.DataFrame(dict(zip(['BKG_DATE','BKG_HOUR','ITEM_QTY'],[(pd.to_datetime('2021-06-28'),pd.to_datetime('2021-06-28')), (4,5), (0,0)]))))
data = data.sort_values(['BKG_DATE','BKG_HOUR'])

comb_date = []
for date, hour, _ in data.values:
    comb_date.append(pd.to_datetime(f'{date.year}-{date.month}-{date.day} {hour}:00:00'))
data['DATE'] = comb_date
data = data.set_index('DATE').drop(columns=['BKG_DATE', 'BKG_HOUR']).rename(columns={'ITEM_QTY':'TARGET'})

1. ACF, PACF가 stationary하도록 전처리
2. seasonality 있다면 SARIMA P,D,Q,M을 정해
3. seasonality를 제외한 ACF, PACF살펴보고 이를 통해 ARIMA p,d,q 결정

Raw Data

In [ ]:
decomposition = seasonal_decompose(data, model='additive')
fig = decomposition.plot()
fig.set_size_inches(20,10)
plt.show()

Log Transform

In [ ]:
data = np.log1p(data)

train = data.iloc[:-720]
val = data.iloc[-720:-360]
test = data.iloc[-360:]

In [ ]:
data.plot(figsize=(10,4))
plt.show()

In [ ]:
decomposition = seasonal_decompose(data, model='additive')
fig = decomposition.plot()
fig.set_size_inches(20,10)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
fig.suptitle('Differenced Data')
plot_acf(data, lags=100, ax=ax[0])
plot_pacf(data, lags=100, ax=ax[1])
plt.show()
# 24차수마다 acf 반복 -> 24 계절 차분

Seasonal Differenced

In [ ]:
data_sdiff = data.diff(24).dropna()

In [ ]:
data_sdiff.plot(figsize=(10,4))
plt.show()

In [ ]:
decomposition = seasonal_decompose(data_sdiff, model='additive')
fig = decomposition.plot()
fig.set_size_inches(20,10)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
fig.suptitle('Differenced Data')
plot_acf(data_sdiff, lags=100, ax=ax[0])
plot_pacf(data_sdiff, lags=100, ax=ax[1])
plt.show()

Seasonal+Trend Differenced

In [ ]:
data_sdiff2 = data.diff(24).diff().dropna()

In [ ]:
data_sdiff2.plot(figsize=(10,4))
plt.show()

In [ ]:
decomposition = seasonal_decompose(data_sdiff2, model='additive')
fig = decomposition.plot()
fig.set_size_inches(20,10)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
fig.suptitle('Differenced Data')
plot_acf(data_sdiff2, lags=100, ax=ax[0])
plot_pacf(data_sdiff2, lags=100, ax=ax[1])
plt.show()

# p = 2, q = 1, P = 1, Q = 1

## Modeling

In [ ]:
model = SARIMAX(data, order=(2,1,1), seasonal_order=(2,1,1,24))
model = model.fit()

In [ ]:
model.summary()

In [ ]:
model.plot_diagnostics(figsize=(16,10))
plt.show()

In [ ]:
pred_list = []
for i in tqdm(range(0,30)):
    date = [pd.to_datetime('2021-05-31 00:00:00') + pd.Timedelta(hours=h) for h in range(744)][0+24*i:48+24*i]
    pred = model.get_prediction(start = date[0], end = date[-1], dynamic=True)
    pred_value = pred.predicted_mean[-24:]
    pred_ci = pred.conf_int()
    pred_list.append((pred_value, pred_ci))

In [ ]:
pred_values = []
pred_cis = []
for value, ci in pred_list:
    pred_values.append(value)
    pred_cis.append(ci)

In [ ]:
pred_value = pd.concat(pred_values)
pred_ci = pd.concat(pred_cis)

In [ ]:
mean_squared_error(np.expm1(pred_value), np.expm1(pd.concat([val,test])), squared=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
np.expm1(pred_value).plot(ax=ax, label='pred', color='k')
np.expm1(pd.concat([val,test])).plot(ax=ax, label='real', color='pink')

ax.fill_between(pred_ci.index, np.expm1(pred_ci.iloc[:,0]), np.expm1(pred_ci.iloc[:,1]),
                color='gray', alpha=0.25, label='ci')
plt.title('SARIMA Prediction')
plt.legend()
plt.show()

In [ ]:
model.save('sarima24.pkl')

In [ ]:
np.expm1(pred_value).reset_index().rename(columns={'index':'DATE',0:'preds'}).to_csv('results_sarima.csv', index=False)